# <a href="https://miptstats.github.io/courses/ad_mipt.html">Phystech@DataScience</a>
## Домашнее задание 7

**Правила, <font color="red">прочитайте внимательно</font>:**

* Выполненную работу нужно отправить телеграм-боту `@miptstats_pds_bot`. Для начала работы с ботом каждый раз отправляйте `/start`. **Работы, присланные иным способом, не принимаются.**
* Дедлайн см. в боте. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Прислать нужно ноутбук в формате `ipynb`.
* Выполнять задание необходимо полностью самостоятельно. **При обнаружении списывания все участники списывания будут сдавать устный зачет.**
* Решения, размещенные на каких-либо интернет-ресурсах, не принимаются. Кроме того, публикация решения в открытом доступе может быть приравнена к предоставлении возможности списать.
* Для выполнения задания используйте этот ноутбук в качестве основы, ничего не удаляя из него. Можно добавлять необходимое количество ячеек.
* Комментарии к решению пишите в markdown-ячейках.
* Выполнение задания (ход решения, выводы и пр.) должно быть осуществлено на русском языке.
* Если код будет не понятен проверяющему, оценка может быть снижена.
* Никакой код из данного задания при проверке запускаться не будет. *Если код студента не выполнен, недописан и т.д., то он не оценивается.*
* **Код из рассказанных на занятиях ноутбуков можно использовать без ограничений.**

**Правила оформления теоретических задач:**

* Решения необходимо прислать одним из следующих способов:
  * фотографией в правильной ориентации, где все четко видно, а почерк разборчив,
    * отправив ее как файл боту вместе с ноутбуком *или*
    * вставив ее в ноутбук посредством `Edit -> Insert Image` (<font color="red">фото, вставленные ссылкой, не принимаются</font>);
  * в виде $\LaTeX$ в markdown-ячейках.
* Решения не проверяются, если какое-то требование не выполнено. Особенно внимательно все проверьте в случае выбора второго пункта (вставки фото в ноутбук). <font color="red"><b>Неправильно вставленные фотографии могут не передаться при отправке.</b></font> Для проверки попробуйте переместить `ipynb` в другую папку и открыть его там.
* В решениях поясняйте, чем вы пользуетесь, хотя бы кратко. Например, если пользуетесь независимостью, то достаточно подписи вида "*X и Y незав.*"
* Решение, в котором есть только ответ, и отсутствуют вычисления, оценивается в 0 баллов.

**Баллы за задание:**

* Задача 1 &mdash;  35 баллов
* Задача 2 &mdash;  30 баллов






-----

In [ ]:
# Bot check

# HW_ID: phds_hw6
# Бот проверит этот ID и предупредит, если случайно сдать что-то не то.

# Status: not final
# Перед отправкой в финальном решении удали "not" в строчке выше.
# Так бот проверит, что ты отправляешь финальную версию, а не промежуточную.
# Никакие значения в этой ячейке не влияют на факт сдачи работы.

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as sps

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.tree import DecisionTreeRegressor

from catboost import CatBoostRegressor, Pool

from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.utils import shuffle

plt.rcParams['axes.facecolor'] = 'lightgrey'
sns.set(palette='Set2', font_scale=1.6)

# Применения градиентного бустинга.

Градиентный бустинг может быть применен в огромном множестве реальных задач.

Реальные задачи, где применяется бустинг:    

☣ 

[Бустинг в предсказании лекарств](https://jcheminf.biomedcentral.com/articles/10.1186/s13321-017-0209-z)

[Предсказание сайтов связывания белков с ДНК](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-018-2527-1)

[Оценивание качества молекулярного докинга](https://pubmed.ncbi.nlm.nih.gov/30873528/)

[Бустинг и нейросети в задаче оценивания влияния вариаций в некодирующих регионах ](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4768299/)

⚛

[Smuon collider analysis](https://arxiv.org/abs/2109.11815): применяется бустинг, PCA, отбор признаков.


[Бустинг и бозон Хиггса](http://proceedings.mlr.press/v42/chen14.pdf)


🏫

[Предсказание распределения студентов](https://www.researchgate.net/publication/338853863_STUDENT_PLACEMENT_PREDICTION_MODEL_USING_GRADIENT_BOOSTED_TREE_ALGORITHM)


[Моделирование индивидуального профиля студента](https://www.igi-global.com/article/student-profile-modeling-using-boosting-algorithms/284084)

# Задача 1

**Внимание!** 

Не забудьте, что на тестовой выборке, по которой делаются итоговые выводы, ничего не должно обучаться.

*Совет. При отладке кода используйте небольшую часть данных. Финальные вычисления проведите на полных данных. Для оценки оставшегося времени работы используйте tqdm в циклах.*

Для исследования вам предлагается несколько датасетов:

### 1. **Профиль биология:** [Датасет](https://archive.ics.uci.edu/ml/datasets/Physicochemical+Properties+of+Protein+Tertiary+Structure) про физикохимические свойства третичной структуры белков.

В этом датасете содержатся следующие признаки:
```
 RMSD-Size of the residue.
 F1 - Total surface area.
 F2 - Non polar exposed area.
 F3 - Fractional area of exposed non polar residue.
 F4 - Fractional area of exposed non polar part of residue.
 F5 - Molecular mass weighted exposed area.
 F6 - Average deviation from standard exposed area of residue.
 F7 - Euclidian distance.
 F8 - Secondary structure penalty.
 F9 - Spacial Distribution constraints (N,K Value).
```




Мы будем предсказывать размер остатка (RMSD) по остальным признакам. 
Датасет был получен с помощью [CASP9](https://predictioncenter.org/CASP9/).




Скачать датасет можно с помощью `sklearn`:

In [ ]:
from sklearn.datasets import fetch_openml

protein = fetch_openml(name='physicochemical-protein')
data = pd.DataFrame(protein.data, columns=protein.feature_names)
X, y = data.drop("RMSD", axis=1), data.RMSD

Этот датасет большой по размеру, так что можете взять в качестве данных его случайную половину.

### 2. **Профиль физика:**  [Датасет](https://archive.ics.uci.edu/ml/datasets/MAGIC+Gamma+Telescope) про излучение Черенкова.


Не забудьте назвать колонки в соотсветствии с признаками (см. описание датасета). Целевой меткой является переменная `class`. Преобразуйте метки классов в числа для корректной работы бустинга.

### 1.

Повторите исследование, проведенное в предыдущем домашнем задании, используя градиентный бустинг из `sklearn`. Сравните полученные результаты со случайным лесом. 

Вкратце опишем задания:
1. Посмотрите, как меняется качество модели в зависимости от значений гиперпараметров и сделайте выводы по результатам. Гиперпараметры:
    * количество деревьев (`n_estimators`) [от 1 до 400];
    * максимальная глубина дерева (`max_depth`) [от 3 до 15]. 
    * скорость обучения (`learning_rate`), поясните также смысл этого параметра.
2. Запустите модель три раза с параметрами по умолчанию. Объясните результаты.
3. Подберите оптимальные гиперпараметры, рассмотренные в первом пункте и зафиксируйте их.
4. Исследуйте зависимость метрики от количества признаков, по которым происходит разбиение в вершине дерева. Объясните результаты.
5. Исследуйте время работы фита и предикта, среднюю глубину деревьев и количество листьев в дереве.

*Замечания.*
* В качестве основы можно использовать свое решение предыдущего задания. В большинстве случаев нужно только заменить `RandomForestRegressor` на `GradientBoostingRegressor`.
* При сравнении методов по одинаковым свойствам желательно рисовать результаты на одном графике.
* Обратите внимание на метод `staged_predict` у `GradientBoostingRegressor`. Он позволяет получить "кумулятивные" предсказания, то есть по первым $t$ деревьям по всем значениям $t$.
* При желании, границы рассмотрения значений гиперпараметров можно увеличить.
* При подборе гиперпараметров при помощи рандомизированного поиска проводите достаточное количество на большой сетке параметров &mdash; как минимум 50. Даже если долго обучается. 
* Используйте `tqdm` для оценки времени выполнения кода. 

### 2. 

Выберите самый значимый признак согласно `feature_importances_`. Визуализируйте работу бустинга, обученного только на одном этом признаке.

Для этого рассмотрите процесс построения бустинга для первых 10 итераций. При визуализации требуется для каждой итерации нарисовать:
* предсказания новой базовой модели и выборку, на которой эта базовая модель обучается;
* предсказания текущей композиции и исходную обучающую выборку. 

Пример графиков смотрите в лекции в разделе бустинг в задаче регресии.

# Задача 2


Возьмем данные из предыдущего дз.

Исследуйте зависимость качества по метрике MAPE для моделей XGBoost, LightGBM,
CatBoost, а так же градиентного бустинга из sklearn, в зависимости от количества
деревьев, их максимальной глубины, шага обучения, а также различных регуляризаций.  При исследовании одного гиперпараметра рисуйте один график для всех моделей. Тщательно подберите цвета и тип линий так, чтобы картинка была легко читаемой. При проведении исследований не забывайте писать подробные комментарии
и выводы.

*Совет. Изучите ноутбук про данные фреймворки.*


В качестве данных возьмём следующие датасеты:

**Профиль физика:** <https://www.kaggle.com/datasets/khsamaha/solar-flares-rhessi>. Будем предсказывать длительность солнечных вспышек. Используйте файл с данными до 2018 года.

**Профиль биология:** <https://archive.ics.uci.edu/dataset/1/abalone>. Будем предсказывать возраст моллюсков.

*Совет.* При отладке кода используйте небольшую часть данных. Финальные вычисления проведите на полных данных. Для оценки времени работы используйте `tqdm` в циклах.

#### Загрузка и предобработка данных:

**Физика:**

In [ ]:
data = pd.read_csv('hessi.solar.flare.UP_To_2018.csv')

Уберем категориальные признаки, а также дату и время начала и время конца вспышки:

In [ ]:
data = data.drop(['flare', 'start.date', 'start.time', 'end', 'flag.1', 'flag.2', 'flag.3', 'flag.4', 'flag.5'], axis=1)
X = data.drop('duration.s', axis=1)
y = data['duration.s']

**Биология:**

In [ ]:
data = pd.read_csv('abalone.data', names=['Sex', 'Length', 'Diameter', 'Height', 'Whole weight', 
                                          'Shucked weight', 'Viscera weight', 'Shell weight',
                                          'Rings'])

Уберем категориальный признак:

In [ ]:
data = data.drop('Sex', axis=1)
X = data.drop('Rings', axis=1)
y = data['Rings'] + 1.5